In [1]:
from stark_qa import load_qa, load_skb
from stark_qa.tools.process_text import normalize_answer

/Users/louislacombe/miniconda3/envs/stark-qa/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_name = 'prime'

# Load the retrieval dataset
qa_dataset = load_qa(dataset_name, human_generated_eval=True)
idx_split = qa_dataset.get_idx_split()

# Load the semi-structured knowledge base
skb = load_skb(dataset_name, download_processed=False, root='.')

Use file from /Users/louislacombe/.cache/huggingface/hub/datasets--snap-stanford--stark/snapshots/7b0352c7dcefbf254478c203bcfdf284a08866ac/qa/prime/stark_qa/stark_qa_human_generated_eval.csv.
Loading from /Users/louislacombe/CODE/quantmetry/stark/prime/processed!


In [3]:
def check_word_in_text(word, text):
    return word in text

def check_similarity_question_answer(question, list_answers, show=False):
    question_ = normalize_answer(question)
    for answer in list_answers:
        answer_ = normalize_answer(answer)
        if check_word_in_text(answer_, question_):
            if show:
                print('Answer:', answer)
                print('Question:', question)
                print("-"*10)
            return True
    return False


def check_questions(qa_dataset, max_number_answers=5, show=False):
    exclude_questions = []
    for item in qa_dataset:
        question_ = item[0]
        list_answer_ = item[2]

        list_answer_names_ = []
        for answer_ in list_answer_:
            list_answer_names_.append(skb.__getitem__(int(answer_)).name)

        if len(list_answer_) > max_number_answers:
            exclude_questions.append(item[1])
            if show:
                print('Too many answers:', list_answer_names_)
                print('Question:', question_)
                print("-"*10)

        # Check if one of the answers is in the questions using regex
        if check_similarity_question_answer(question_, list_answer_names_, show=show):
            exclude_questions.append(item[1])

    exclude_questions = list(set(exclude_questions))
    if show:
        print('Number of questions to exclude:', len(exclude_questions))

    qa_dataset_filtered = []
    for item in qa_dataset:
        if item[1] not in exclude_questions:
            qa_dataset_filtered.append(item)
    return qa_dataset_filtered



In [4]:
filtered_questions = check_questions(qa_dataset, show=True)

Answer: mixed mucinous and nonmucinous bronchioloalveolar adenocarcinoma
Question: mixed mucinous and nonmucinous bronchioloalveolar adenocarcinoma is a subtype of what disease?
----------
Too many answers: ['(1H-indol-3-yl)-(2-mercapto-ethoxyimino)-acetic acid', 'AICA ribonucleotide', 'Triptolide', 'Teriflunomide', 'R-348', 'Acteoside', 'Roquinimex']
Question: What drug with ~300 Da weight that stimulates the immune system has synergy with a few different acids?
----------
Too many answers: ['scyllo-inositol', 'D-glucitol 6-phosphate', 'Human blood group H type 1 trisaccharide', 'Sucrose', '1-3 Sugar Ring of Pentamannosyl 6-Phosphate', 'Acarbose Derived Hexasaccharide']
Question: What variant of sugar (yum!) hits a protein involved in carbohydrate binding?
----------
Too many answers: ['bestiality', 'pedophilia', 'exhibitionism', 'psychosexual disorder', 'fetishism', 'voyeurism']
Question: What type of disease is a paraphilic disorder?
----------
Too many answers: ['migraine with aura